In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import boto3
import os
from io import StringIO
from datetime import datetime

date = datetime.now().strftime('%d%m%Y')
CLEANED_FILE = f"full_stg_extract_cleaned_{date}.csv"
OUTPUT_FILE = "machine_learning/data/full/" + CLEANED_FILE

load_dotenv()
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY_ID')
)
s3_client = session.client('s3')
BUCKET_NAME = os.getenv('BUCKET_NAME')

# Obtenemos los últimos archivos de cada una de las fuentes
def extract_date(file):
    date_part = file.split('_')[-1].replace('.csv', '')  # Get "04022025"
    return pd.to_datetime(date_part, format="%d%m%Y")  # Convert to datetime

# Empezamos por ZonaProp
files = s3_client.list_objects_v2(Bucket=BUCKET_NAME)
sub_files = list()
for file in files["Contents"]:
    if ("/ZonaProp/STG" in file["Key"]) and (".csv" in file["Key"]):
        sub_files.append(file["Key"])

sorted_files = sorted(sub_files, key=extract_date, reverse=True)
zonaprop_file = sorted_files[:1][0]
print("Using zonaprop file {}".format(zonaprop_file))

for file in files["Contents"]:
    if ("/ArgenProp/STG" in file["Key"]) and (".csv" in file["Key"]):
        sub_files.append(file["Key"])

sorted_files = sorted(sub_files, key=extract_date, reverse=True)
argenprop_file = sorted_files[:1][0]
print("Using argenprop file {}".format(argenprop_file))

response = s3_client.get_object(Bucket=BUCKET_NAME, Key=zonaprop_file)
csv_data = response['Body'].read().decode('utf-8')  # Convert bytes to string
df_zonaprop = pd.read_csv(StringIO(csv_data))

response = s3_client.get_object(Bucket=BUCKET_NAME, Key=argenprop_file)
csv_data = response['Body'].read().decode('utf-8')  # Convert bytes to string
df_argenprop = pd.read_csv(StringIO(csv_data))

df = pd.concat([df_zonaprop, df_argenprop], ignore_index=True)

Using zonaprop file scrapping/ZonaProp/STG/STG_ZonaProp_12022025.csv
Using argenprop file scrapping/ArgenProp/STG/STG_ArgenProp_13022025.csv


In [13]:
# Debido a que no nos coinciden los barrios de la info histórica con los de ZonaProp, vamos a hacer un mapeo
# y normalización.

import pandas as pd
from difflib import get_close_matches

# Lista de valores permitidos
valid_neighborhoods = [
    'ALMAGRO', 'BALVANERA', 'BELGRANO', 'CABALLITO', 'COLEGIALES', 'DEVOTO',
    'FLORES', 'MONTSERRAT', 'NUNEZ', 'PALERMO', 'PARQUE PATRICIOS', 'PUERTO MADERO',
    'RECOLETA', 'RETIRO', 'SAN NICOLAS', 'SAN TELMO', 'VILLA CRESPO', 'VILLA DEL PARQUE', 'VILLA URQUIZA'
]

# Función para mapear los barrios
def map_neighborhood(neighborhood):
    neighborhood = neighborhood.upper()  # Convertimos a mayúsculas para estandarizar
    
    # Mapeos directos conocidos
    manual_mappings = {
        'MONSERRAT': 'MONTSERRAT',
        'NUÑEZ': 'NUNEZ',
        'CONGRESO': 'BALVANERA',
        'BARRIO NORTE': 'RECOLETA',
        'TRIBUNALES': 'SAN NICOLAS',
        'MICROCENTRO': 'SAN NICOLAS',
        'CENTRO / MICROCENTRO': 'SAN NICOLAS',
        'BARRACAS': 'PARQUE PATRICIOS',
        'CONSTITUCIÓN': 'SAN TELMO',
        'POMPEYA': 'PARQUE PATRICIOS',
        'MATADEROS': 'FLORES',
        'LINIERS': 'FLORES',
        'VERSALLES': 'VILLA URQUIZA',
        'VILLA SOLDATI': 'PARQUE PATRICIOS',
        'VILLA RIACHUELO': 'PARQUE PATRICIOS',
        'VILLA LUGANO': 'PARQUE PATRICIOS',
        'LA PATERNAL': 'VILLA CRESPO',
        'COGHLAN': 'BELGRANO',
        'LAS CAÑITAS': 'BELGRANO',
        'VILLA PUEYRREDÓN': 'VILLA URQUIZA',
        'BOTÁNICO': 'PALERMO',
        'BOEDO': 'ALMAGRO',
        'CHACARITA': 'VILLA CRESPO',
        'PALERMO HOLLYWOOD': 'PALERMO',
        'SAN CRISTOBAL': 'BALVANERA',
        'SAAVEDRA': 'NUNEZ',
        'AGRONOMÍA': 'VILLA DEL PARQUE',
        'BARRIO PARQUE': 'PALERMO',
        'ONCE': 'BALVANERA',
        'BARRIO CHINO': 'BELGRANO',
        'LOMAS DE NÚÑEZ': 'NUNEZ',
        'LA BOCA': 'SAN TELMO',
        'MONTE CASTRO': 'DEVOTO',
        'CID CAMPEADOR': 'VILLA CRESPO',
        'BARRIO PARQUE GENERAL BELGRANO': 'BELGRANO',
        'PRIMERA JUNTA': 'CABALLITO',
        'VELEZ SARSFIELD': 'FLORES',
        'PARQUE AVELLANEDA': 'FLORES',
        'DISTRITO QUARTIER': 'PUERTO MADERO',
        'TEMPERLEY': 'FLORES',  # No está en CABA, asignado al más cercano
        'CATALINAS': 'RETIRO',
        'LOS PERALES': 'FLORES',
        'NAÓN': 'VILLA DEL PARQUE',
        'PATERNAL': 'VILLA CRESPO',
        'BOCA': 'SAN TELMO',
        'CONSTITUCION': 'SAN TELMO',
        'DEPARTAMENTO EN VENTA EN CABALLITO': 'CABALLITO',
        'DEPARTAMENTO EN VENTA EN PALERMO': 'PALERMO',
        'AGRONOMIA': 'VILLA DEL PARQUE'
    }
    
    if neighborhood in manual_mappings:
        return manual_mappings[neighborhood]
    
    # Buscar coincidencias aproximadas
    match = get_close_matches(neighborhood, valid_neighborhoods, n=1, cutoff=0.6)
    return match[0] if match else 'OTRO'

In [14]:
df['normalized_neighborhood'] = df['neighborhood'].apply(map_neighborhood)
df.head()


,property_url,zonaprop_code,price,expenses,address,location,description,neighborhood,total_area,rooms,bedrooms,bathrooms,garages,antiquity,argenprop_code,normalized_neighborhood
0,https://www.zonaprop.com.ar/propiedades/clasif...,55432076.0,4800000.0,1,NaN,Capital Federal,Contacto: G A Ver datos Excelente departamento...,Belgrano,125.0,3.0,2.0,3.0,1.0,NaN,NaN,BELGRANO
1,https://www.zonaprop.com.ar/propiedades/clasif...,55525585.0,2940000.0,500.000,NaN,Capital Federal,Alquiler Semipiso de 5 amb ubicado en Belgrano...,Palermo,189.0,5.0,3.0,3.0,1.0,NaN,NaN,PALERMO
2,https://www.zonaprop.com.ar/propiedades/clasif...,55469522.0,420.000,97.000,NaN,Capital Federal,Alquiler vivienda o comercialedificio ubicado ...,Tribunales,44.0,2.0,1.0,1.0,NaN,NaN,NaN,SAN NICOLAS
3,https://www.zonaprop.com.ar/propiedades/clasif...,55264598.0,1440000.0,300.000,NaN,Belgrano,Excelente departamento en esquina con vistas m...,Belgrano R,105.0,4.0,3.0,2.0,1.0,NaN,NaN,BELGRANO
4,https://www.zonaprop.com.ar/propiedades/clasif...,55401193.0,590.000,70.000,NaN,Capital Federal,"Departamento 3 ambientes, todo en muy buen est...",La Paternal,50.0,3.0,2.0,1.0,NaN,NaN,NaN,VILLA CRESPO


In [15]:
df.loc[df["normalized_neighborhood"] == "OTRO"]

,property_url,zonaprop_code,price,expenses,address,location,description,neighborhood,total_area,rooms,bedrooms,bathrooms,garages,antiquity,argenprop_code,normalized_neighborhood
